In [1]:
import os
import pickle

# From Keras

In [2]:
!pip install keras

You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import keras
print("keras version: {}".format(keras.__version__))

Using TensorFlow backend.


keras version: 2.2.4


In [4]:
NUM_WORDS=1000 # only use top 1000 words
INDEX_FROM=3   # word index offset

if os.path.exists('/notebooks/data'):
    k_train, k_test = keras.datasets.imdb.load_data(path='/notebooks/data/imdb/imdb.npz', num_words=NUM_WORDS, index_from=INDEX_FROM)
elif os.path.exists('/Volumes/Data'):
    k_train, k_test = keras.datasets.imdb.load_data(path='/Volumes/Data/imdb/imdb.npz', num_words=NUM_WORDS, index_from=INDEX_FROM)
else:
    raise IOError("no such path")
    
k_train_x, k_train_y = k_train
k_test_x, k_test_y = k_test

## Show origin text

In [5]:
word_to_id = keras.datasets.imdb.get_word_index()
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2

id_to_word = {value:key for key,value in word_to_id.items()}
print(' '.join(id_to_word[id] for id in k_train_x[0] ))

<START> this film was just brilliant casting <UNK> <UNK> story direction <UNK> really <UNK> the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same <UNK> <UNK> as myself so i loved the fact there was a real <UNK> with this film the <UNK> <UNK> throughout the film were great it was just brilliant so much that i <UNK> the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the <UNK> <UNK> was amazing really <UNK> at the end it was so sad and you know what they say if you <UNK> at a film it must have been good and this definitely was also <UNK> to the two little <UNK> that played the <UNK> of <UNK> and paul they were just brilliant children are often left out of the <UNK> <UNK> i think because the stars that play them all <UNK> up are such a big <UNK> for the whole film but these children are amazing and should be <UNK> for what they have done don't yo

## show train data

In [6]:
print("train data count: {}".format(len(k_train_x)))
print(k_train_x[0])

train data count: 25000
[1, 14, 22, 16, 43, 530, 973, 2, 2, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 2, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 2, 2, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 2, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 2, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 2, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 2, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32]


## Data Preprocessing

In [7]:
# expand sequence to length of 500 with value = 0 if the sequence length <= 500
# truncated sequence to length of 500 if the sequence length > 500
k_train_x_p = keras.preprocessing.sequence.pad_sequences(k_train_x, maxlen=500, value=0., padding="post")
k_test_x_p = keras.preprocessing.sequence.pad_sequences(k_test_x, maxlen=500, value=0., padding="post")

In [8]:
# a simple example
k_train_x_p[0]

array([  1,  14,  22,  16,  43, 530, 973,   2,   2,  65, 458,   2,  66,
         2,   4, 173,  36, 256,   5,  25, 100,  43, 838, 112,  50, 670,
         2,   9,  35, 480, 284,   5, 150,   4, 172, 112, 167,   2, 336,
       385,  39,   4, 172,   2,   2,  17, 546,  38,  13, 447,   4, 192,
        50,  16,   6, 147,   2,  19,  14,  22,   4,   2,   2, 469,   4,
        22,  71,  87,  12,  16,  43, 530,  38,  76,  15,  13,   2,   4,
        22,  17, 515,  17,  12,  16, 626,  18,   2,   5,  62, 386,  12,
         8, 316,   8, 106,   5,   4,   2,   2,  16, 480,  66,   2,  33,
         4, 130,  12,  16,  38, 619,   5,  25, 124,  51,  36, 135,  48,
        25,   2,  33,   6,  22,  12, 215,  28,  77,  52,   5,  14, 407,
        16,  82,   2,   8,   4, 107, 117,   2,  15, 256,   4,   2,   7,
         2,   5, 723,  36,  71,  43, 530, 476,  26, 400, 317,  46,   7,
         4,   2,   2,  13, 104,  88,   4, 381,  15, 297,  98,  32,   2,
        56,  26, 141,   6, 194,   2,  18,   4, 226,  22,  21, 13

In [9]:
k_train_x_p_c = keras.utils.to_categorical(k_train_y, num_classes=2, dtype='int')
k_test_x_p_c = keras.utils.to_categorical(k_test_y, num_classes=2, dtype='int')

In [10]:
k_train_x_p_c[0]

array([0, 1])

# From tflearn

In [11]:
!pip install tflearn

You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [12]:
import tflearn

!pip list | grep 'tflearn'

tflearn             0.3.2                 
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [13]:
from tflearn.data_utils import to_categorical, pad_sequences
from tflearn.datasets import imdb

In [14]:
# n_word=30000, most common used 300000 word
if os.path.exists('/notebooks/data'):
    t_train, t_test, _ = imdb.load_data('/notebooks/data/imdb/imdb.pkl', n_words=1000, valid_portion=0.)
elif os.path.exists('/Volumes/Data'):
    t_train, t_test, _ = imdb.load_data('/Volumes/Data/imdb/imdb.pkl', n_words=1000, valid_portion=0.)
else:
    raise IOError("no such path")

# trainX, testX: the sequence
# trainY, testY: the emotion label (positive/negative, 1/0)
t_train_x, t_train_y = t_train
t_test_x, t_test_y = t_test

## Show train data

In [15]:
print("train data count: {}".format(len(t_train_x)))
print(t_train[0][23000])

train data count: 25000
[6, 1, 1, 1, 29, 1, 28, 44, 6, 1, 7, 55, 947, 938, 854, 1, 1, 1, 7, 1, 7, 721, 170, 47, 712, 8, 815, 15, 31, 6, 1, 1, 61, 1, 170, 608, 31, 6, 51, 1, 1, 4, 51, 59, 43, 183, 10, 6, 1, 64, 1, 85, 245, 1, 31, 712, 3, 6, 322, 5, 494, 1, 1, 202, 256, 3, 5, 52, 335, 3, 6, 1, 1, 1, 44, 64, 85, 2, 1, 7, 2, 1, 1, 34, 56, 1, 324, 15, 4, 12, 13, 9, 11, 12, 13, 9, 11, 31, 2, 1, 3, 17, 76, 10, 6, 1, 1, 595, 747, 4, 61, 1, 14, 8, 1, 20, 1, 3, 26, 17, 10, 6, 1, 3, 5, 3, 15, 61, 770, 3, 45, 1, 8, 2, 1, 533, 4, 1, 7, 59, 181, 14, 6, 1, 27, 3, 263, 3, 106, 2, 1, 3, 26, 102, 125, 1, 7, 122, 63, 30, 150, 159, 733, 55, 60, 12, 13, 9, 11, 12, 13, 9, 11, 1, 3, 17, 10, 6, 1, 803, 55, 1, 5, 1, 5, 2, 1, 1, 1, 23, 106, 1, 5, 1, 372, 4, 261, 2, 27, 145, 1, 24, 41, 2, 1, 23, 275, 709, 18, 81, 37, 229, 8, 539, 47, 300, 49, 345, 1, 4, 12, 13, 9, 11, 12, 13, 9, 11, 1, 1, 10, 33, 2, 75, 296, 152, 17, 27, 10, 134, 1, 22, 322, 4, 2, 330, 323, 3, 5, 68, 2, 1, 7, 2, 1, 35, 979, 211, 8, 112, 2, 1, 94

# Data Preprocessing

In [16]:
# expand sequence to length of 500 with value = 0 if the sequence length <= 500
# truncated sequence to length of 500 if the sequence length > 500
t_train_x_p = pad_sequences(t_train_x, maxlen=500, value=0.)
t_test_x_p = pad_sequences(t_test_x, maxlen=500, value=0.)

In [17]:
# a simple example
t_train_x_p[23000]

array([  6,   1,   1,   1,  29,   1,  28,  44,   6,   1,   7,  55, 947,
       938, 854,   1,   1,   1,   7,   1,   7, 721, 170,  47, 712,   8,
       815,  15,  31,   6,   1,   1,  61,   1, 170, 608,  31,   6,  51,
         1,   1,   4,  51,  59,  43, 183,  10,   6,   1,  64,   1,  85,
       245,   1,  31, 712,   3,   6, 322,   5, 494,   1,   1, 202, 256,
         3,   5,  52, 335,   3,   6,   1,   1,   1,  44,  64,  85,   2,
         1,   7,   2,   1,   1,  34,  56,   1, 324,  15,   4,  12,  13,
         9,  11,  12,  13,   9,  11,  31,   2,   1,   3,  17,  76,  10,
         6,   1,   1, 595, 747,   4,  61,   1,  14,   8,   1,  20,   1,
         3,  26,  17,  10,   6,   1,   3,   5,   3,  15,  61, 770,   3,
        45,   1,   8,   2,   1, 533,   4,   1,   7,  59, 181,  14,   6,
         1,  27,   3, 263,   3, 106,   2,   1,   3,  26, 102, 125,   1,
         7, 122,  63,  30, 150, 159, 733,  55,  60,  12,  13,   9,  11,
        12,  13,   9,  11,   1,   3,  17,  10,   6,   1, 803,  5

In [18]:
# change label to one-hot encoding
t_train_y_c = to_categorical(t_train_y, nb_classes=2)
t_test_y_c = to_categorical(t_test_y, nb_classes=2)

In [19]:
print(t_train_y_c[0])

[1. 0.]


# Using Dataset

In [20]:
# data loading method
class IMDBDataset():
    def __init__(self, X, Y):
        self.data_count = len(X)
        self.input = X
        self.label = Y
        self.ptr = 0
        
    def minibatch(self, size):
        ret = None
        
        if self.ptr + size < self.data_count:
            ret = self.input[self.ptr:(self.ptr + size)], self.label[self.ptr:(self.ptr + size)]
        else:
            ret = np.concatenate((self.input[self.ptr:], self.input[:size-len(self.input[self.ptr:])])), \
                  np.concatenate((self.label[self.ptr:], self.label[:size-len(self.label[self.ptr:])]))
        
        self.ptr = (self.ptr + size) % (self.data_count)
        return ret